[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/latent-consistency-model-colab/blob/main/lcm_img2img_colab.ipynb)

In [ ]:
!pip install -q diffusers transformers accelerate peft gradio==3.50.2

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7", revision="fb9c5d167af11fd84454ae6493878b10bb63b067", safety_checker=None, custom_pipeline="latent_consistency_img2img")
pipe.to(torch_device="cuda", torch_dtype=torch.float16)
pipe.set_progress_bar_config(disable=True)

In [ ]:
import gradio as gr

def generate(prompt, input_image):
  image = pipe(prompt, image=input_image, num_inference_steps=4, guidance_scale=8.0, lcm_origin_steps=50, strength=0.8).images[0]
  return image.resize((768, 768))

with gr.Blocks(css="style.css", title=f"Realtime Latent Consistency Model") as demo:
    with gr.Row():
      with gr.Column(scale=23):
          prompt_box = gr.Textbox(show_label=False, value="a close-up picture of a fluffy cat standing in the rain")

    with gr.Row(variant="default"):
        input_image = gr.Image(
            show_label=False,
            type="pil",
            tool="color-sketch",
            source="canvas",
            height=742,
            width=742,
            brush_radius=20.0,
        )
        output_image = gr.Image(
            show_label=False,
            type="pil",
            interactive=False,
            height=742,
            width=742,
            elem_id="output_image",
        )

    input_image.change(fn=generate, inputs=[prompt_box, input_image], outputs=[output_image], show_progress=False)

demo.launch(inline=False, share=True)